In [0]:
from pyspark.sql import SparkSession

In [0]:
spark= SparkSession.builder.appName('build').getOrCreate()

In [0]:
df=spark.read.csv('/FileStore/tables/College.csv',inferSchema=True,header=True)

In [0]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, StringIndexer

df.columns

Out[4]: ['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [0]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [0]:
output = assembler.transform(df)

In [0]:
index =StringIndexer(inputCol='Private',outputCol='Private_indx')
output_fin = index.fit(output).transform(output)

In [0]:
final_df=output_fin.select('features','Private_indx')
train_df,test_df=final_df.randomSplit([0.7,0.3])

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

dt_class=DecisionTreeClassifier(labelCol='Private_indx',featuresCol='features')
rf_class=RandomForestClassifier(labelCol='Private_indx',featuresCol='features')
gbt_class=GBTClassifier(labelCol='Private_indx',featuresCol='features')

df_mod=dt_class.fit(train_df)
rf_mod=rf_class.fit(train_df)
gbt_mod=gbt_class.fit(train_df)

In [0]:
df_pred=df_mod.transform(test_df)
rf_pred=rf_mod.transform(test_df)
gbt_pred=gbt_mod.transform(test_df)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_bin_evaluator=BinaryClassificationEvaluator(labelCol='Private_indx')

print('Decision Tree classifier evaluation')
print(my_bin_evaluator.evaluate(df_pred))
print('Random Forest Tree classifier evaluation')
print(my_bin_evaluator.evaluate(rf_pred))
print('GBT classifier evaluation')
print(my_bin_evaluator.evaluate(gbt_pred))

Decision Tree classifier evaluation
0.9219529579472558
Random Forest Tree classifier evaluation
0.9780628811277418
GBT classifier evaluation
0.9547398431931575